In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split#集成的划分训练集模型
from sklearn.model_selection import cross_val_score#k折交叉验证
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPClassifier#神经网络，分层状态机
from sklearn.preprocessing import StandardScaler

In [2]:
#读取excel
AF_input_frame=pd.read_excel('pre_treatment_data.xlsx',sheet_name='AF_input')
nonAF_input_frame=pd.read_excel('pre_treatment_data.xlsx',sheet_name='nonAF_input')
AF_label_frame=pd.read_excel('pre_treatment_data.xlsx',sheet_name='AF_label')
nonAF_label_frame=pd.read_excel('pre_treatment_data.xlsx',sheet_name='nonAF_label')

#将data_frame转化为numpy的ndarry的格式
AF_input=AF_input_frame.to_numpy()
nonAF_input=nonAF_input_frame.to_numpy()
AF_label=AF_label_frame.to_numpy()
nonAF_label=nonAF_label_frame.to_numpy()

AF_label=AF_label.flatten()#二维转一维，所有值在一维里
nonAF_label=nonAF_label.flatten()

In [3]:
raw_x=np.concatenate([AF_input,nonAF_input])#数据集x
raw_y=np.concatenate([AF_label[0:],nonAF_label[0:]])#标签y

(40480, 6)

In [4]:
#所有特征，未选择提取
model = MLPClassifier(hidden_layer_sizes=(10, 10), max_iter=500)
acc = cross_val_score(model,raw_x, raw_y, cv = 10, scoring='accuracy')
mean_acc = np.array(acc).mean()
mean_acc

0.8638833992094861

In [10]:
#将特征进行归一化，得到全部特征的概率，有显著提升
ss = StandardScaler()
std_x = ss.fit_transform(raw_x)
model = MLPClassifier(hidden_layer_sizes=(10, 10), max_iter=500)
acc = cross_val_score(model,std_x, raw_y, cv = 10, scoring='accuracy')
mean_acc = np.array(acc).mean()
mean_acc

0.8969120553359684

In [15]:
#选择6个特征，并进行归一化
ss = StandardScaler()
std_x1 = ss.fit_transform(raw_x[:,[0,1,2,4,5,7]])

In [14]:
#通过贝叶斯得到的概率，选择了6个重要特征
ss = StandardScaler()
std_x1 = ss.fit_transform(raw_x[:,[0,1,2,4,5,7]])
model = MLPClassifier(hidden_layer_sizes=(10, 10), max_iter=500)
acc = cross_val_score(model,std_x1, raw_y, cv = 10, scoring='accuracy')
mean_acc = np.array(acc).mean()
mean_acc

0.9022974308300394

In [16]:
#进行PCA降维
pca=PCA(n_components=5,whiten='full')
pca.fit(std_x1)
std_x1PCA=pca.transform(std_x1)
std_x1PCA.shape

(40480, 5)

In [25]:
#归一择取后的6个特征，进行降维到3,4,5维的正确率
ans=[]
for i in range(3,6):
    pca=PCA(n_components=i,whiten='full')
    pca.fit(std_x1)
    std_x1PCA=pca.transform(std_x1)
    std_x1PCA.shape
    model = MLPClassifier(hidden_layer_sizes=(10, 10), max_iter=500)
    acc = cross_val_score(model,std_x1PCA, raw_y, cv =10, scoring='accuracy')
    mean_acc = np.array(acc).mean()
    ans.append(mean_acc)
    print(mean_acc)

0.8555335968379445
0.9002470355731225
0.8999752964426879


In [26]:
#对所有归一化的8个特征，降维到4,5,6,7维的正确率
ans=[]
for i in range(4,8):
    pca=PCA(n_components=i,whiten='full')
    pca.fit(std_x)
    std_xPCA=pca.transform(std_x)
    std_xPCA.shape

    model = MLPClassifier(hidden_layer_sizes=(10, 10), max_iter=500)
    acc = cross_val_score(model,std_xPCA, raw_y, cv =10, scoring='accuracy')
    mean_acc = np.array(acc).mean()
    ans.append(mean_acc)
    print(mean_acc)

0.8952569169960475
0.8976778656126483
0.8994565217391305
0.9008399209486166


In [18]:
kf = KFold(n_splits=10, shuffle=True, random_state=1)
#kf = KFold(n_splits=10)

In [19]:
#进行kfold的shuffle为True时的打乱顺序即洗牌的交叉验证
model = MLPClassifier(hidden_layer_sizes=(10, 10), max_iter=500)
acc = cross_val_score(model,std_xPCA, raw_y, cv = kf, scoring='accuracy')
mean_acc = np.array(acc).mean()
mean_acc

0.9370800395256916

In [22]:
#进行kfold的shuffle为True时的打乱顺序即洗牌的交叉验证
model = MLPClassifier(hidden_layer_sizes=(10, 10), max_iter=500)
acc = cross_val_score(model,std_x, raw_y, cv = kf, scoring='accuracy')
mean_acc = np.array(acc).mean()
mean_acc

0.9400197628458498